In [1]:
import torch
import numpy as np
from data import Task1Data, Task2Data, Task3Data
import os
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
norm_dataset = Task1Data()
abide_dataset = Task2Data()
mci_dataset = Task3Data(no=2)
ad_dataset = Task3Data(no=1)


(712, 62, 2) (712,) (712,)
(999, 62, 2) (999,) (999,)
(649, 62, 2) (649,) (649,)
Finding files: 2360/2360/2360
Finding files: 1031/1031/1031
Finding files: 712/712/712
380/332
Finding files: 604/604/604
272/332


In [6]:
name = 'res/search_param0.002_mom0.05/'
all_data = np.zeros((2,100,256,62))

sex_0_mean = np.zeros((100,62))
sex_0_std = np.zeros((100,62))

sex_1_mean = np.zeros((100,62))
sex_1_std = np.zeros((100,62))
for sex in range(2):
    for age in range(100):
        dat = np.load(f"{name}//sex-{sex}_age-{age}.npy")
        if sex == 0:
            sex_0_mean[age] = dat.mean(0).reshape(-1)[:62]
            sex_0_std[age] = dat.std(0).reshape(-1)[:62]
        else:
            sex_1_mean[age] = dat.mean(0).reshape(-1)[:62]
            sex_1_std[age] = dat.std(0).reshape(-1)[:62]
        all_data[sex,age] = dat[:,:62]

In [78]:
name = 'res/search_param0.002_mom0.05/'
all_data = np.zeros((2,100,256,62))

statis_dist = np.zeros((2,100,62, 10))
statis_std = np.zeros((2,100,62, 1))


for sex in range(2):
    for age in range(100):
        dat = np.load(f"{name}//sex-{sex}_age-{age}.npy")
        for bins in range(1,11):
            statis_dist[sex,age,:,bins-1] = np.quantile(dat[:,:62],bins*0.1, axis=0)
        statis_std[sex,age,:,0] = dat.std(0).reshape(-1)[:62]
        all_data[sex,age] = dat[:,:62]

In [124]:
use_feas = abide_dataset.ct_feas[:,:,0]
use_feas = use_feas[...,None].repeat(10,axis=2)
use_age = abide_dataset.ages
use_sex = abide_dataset.sexs
use_dx = abide_dataset.dx

mine_feas = np.zeros_like(use_feas)
mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]

In [137]:
use_feas = ad_dataset.ct_feas[:,:,0]
use_feas = use_feas[...,None].repeat(10,axis=2)
use_age = ad_dataset.ages
use_sex = ad_dataset.sexs
use_dx = ad_dataset.dx

mine_feas = np.zeros_like(use_feas)
mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]

In [164]:
use_feas = mci_dataset.ct_feas[:,:,0]
use_feas = use_feas[...,None].repeat(10,axis=2)
use_age = mci_dataset.ages
use_sex = mci_dataset.sexs
use_dx = mci_dataset.dx

mine_feas = np.zeros_like(use_feas)
mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]

In [165]:
mine_X = mine_feas.reshape((len(mine_feas)), -1)
mine_y = use_dx
z_scores = mine_feas[...,4]


In [166]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 尝试不同的k值
scores = []

kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(mine_X)
z_scores = mine_feas[...,4]

if z_scores[labels == 1].mean() > z_scores[labels == 0].mean():
    labels = 1 - labels
score = accuracy_score(labels, mine_y)
print(score)


0.5154494382022472


In [167]:
from sklearn.mixture import GaussianMixture

# 使用GMM
gmm = GaussianMixture(n_components=2, random_state=42)
labels_gmm = gmm.fit_predict(mine_X)

# 确保标签对应关系
if z_scores[labels_gmm == 1].mean() > z_scores[labels_gmm == 0].mean():
    labels_gmm = 1 - labels_gmm

score_gmm = accuracy_score(labels_gmm, mine_y)
print(f"GMM Accuracy: {score_gmm}")

GMM Accuracy: 0.49297752808988765


In [168]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=50)
labels_dbscan = dbscan.fit_predict(mine_X)

# 将-1标签转换为0或1
if -1 in labels_dbscan:
    labels_dbscan = (labels_dbscan == -1).astype(int)

# 确保标签对应关系
if z_scores[labels_dbscan == 1].mean() > z_scores[labels_dbscan == 0].mean():
    labels_dbscan = 1 - labels_dbscan

score_dbscan = accuracy_score(labels_dbscan, mine_y)
print(f"DBSCAN Accuracy: {score_dbscan}")

DBSCAN Accuracy: 0.5337078651685393


In [174]:
# 如果mine_feas有多个特征，可以结合它们
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# PCA降维
pca = PCA(n_components=0.99)  # 保留95%方差
X_pca = pca.fit_transform(mine_X)

# 使用GMM在降维后的数据上
gmm = GaussianMixture(n_components=2, random_state=42)
labels_pca_gmm = gmm.fit_predict(X_pca)

# 确保标签对应关系
if z_scores[labels_pca_gmm == 1].mean() > z_scores[labels_pca_gmm == 0].mean():
    labels_pca_gmm = 1 - labels_pca_gmm

score_pca_gmm = accuracy_score(labels_pca_gmm, mine_y)
print(f"PCA+GMM Accuracy: {score_pca_gmm}")

PCA+GMM Accuracy: 0.5182584269662921


In [89]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [180]:
svm1 = SVC()
svm2 = SVC()

pipeline = make_pipeline(svm1)
X = use_feas.reshape((len(use_feas), -1))
y = use_dx
# 设置5折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# 计算交叉验证的分数
scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy')

# 输出每折的得分和平均分
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean():.2f}")

print("*" * 40)
X2 = mine_feas.reshape((len(mine_feas), -1))
y = use_dx
# pipeline2 = make_pipeline(svm)
pipeline2 = make_pipeline(svm2)

scores2 = cross_val_score(pipeline2, X2, y, cv=kf, scoring='accuracy')
# 输出每折的得分和平均分
print(f"Cross-validation scores: {scores2}")
print(f"Mean accuracy: {scores2.mean():.2f}")

Cross-validation scores: [0.56643357 0.62237762 0.57746479 0.57746479 0.61971831]
Mean accuracy: 0.59
****************************************
Cross-validation scores: [0.53846154 0.60839161 0.57042254 0.6056338  0.61971831]
Mean accuracy: 0.59
